In [1]:
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install spotipy

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score
from sklearn.metrics import silhouette_score
from sklearn.metrics import classification_report
from sklearn.metrics import pairwise_distances_argmin_min

In [4]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="e5af6275806e4e8ebf14a5d71e202fad",
                                                           client_secret="a72673ed83bb437d83c801b760a066d5"))